In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows=300
pd.options.display.max_columns = None

In [ ]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sub = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

print('train_df=',train_df.shape)
print('test_df=',test_df.shape)

df = pd.concat([train_df,test_df],ignore_index=True,sort=False)
df["Transported"].replace([False,True],[0,1],inplace=True)

In [ ]:
def get_missing(df):
    return [(col,(df[col].isnull().sum()/len(df))*100)
            for col in df.columns.tolist()
            if df[col].isnull().sum()>0]
            
missing_df = get_missing(df)    

mis_df = pd.DataFrame(missing_df,columns=['features','missing_pct']).sort_values(by='missing_pct',ascending=False)
print(mis_df)
corrmap = df.corr()
print('\n <features_correlation> \n',corrmap.nlargest(13,'Transported')['Transported'])

In [ ]:
float_cols = df.select_dtypes(float).columns.tolist()
ob_cols = df.select_dtypes(object).columns.tolist()

plt.figure(figsize=(8,4))
plt.hist(df.loc[df["Transported"]==0,'RoomService'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='0')
plt.hist(df.loc[df["Transported"]==1,'RoomService'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='1')
plt.legend(loc='upper right',title='Transported')
plt.title('relation of Transported & RoomServie',fontsize=18)
plt.grid(ls=':')
plt.xlabel('RoomService')
plt.ylabel('count')

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df.loc[df["Transported"]==0,'VRDeck'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='0')
plt.hist(df.loc[df["Transported"]==1,'VRDeck'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='1')
plt.legend(loc='upper right',title='VRDeck')
plt.title('relation of Transported & VRDeck',fontsize=18)
plt.grid(ls=':')
plt.xlabel('VRDeck')
plt.ylabel('count')

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df.loc[df["Transported"]==0,'Spa'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='0')
plt.hist(df.loc[df["Transported"]==1,'Spa'].dropna(),
         range=(0,2500),bins=25,alpha=0.5,label='1')
plt.legend(loc='upper right',title='Spa')
plt.title('relation of Transported & Spa',fontsize=18)
plt.grid(ls=':')
plt.xlabel('Spa')
plt.ylabel('count')

In [ ]:
df["RoomService"].describe()
df["RoomService"].fillna(df["RoomService"].median(),inplace=True)

df["FoodCourt"].describe()
df["FoodCourt"].fillna(df["FoodCourt"].median(),inplace=True)

df["ShoppingMall"].describe()
df["ShoppingMall"].fillna(df["ShoppingMall"].median(),inplace=True)

df["Spa"].describe()
df["Spa"].fillna(df["Spa"].median(),inplace=True)

df["VRDeck"].describe()
df["VRDeck"].fillna(df["VRDeck"].median(),inplace=True)

df["Fun"] = df["RoomService"]+df["FoodCourt"]+df["ShoppingMall"]+df["Spa"]+df["VRDeck"]

df["Fun"].describe()
df["Fun"].fillna(df["Fun"].median(),inplace=True)

df["CryoSleep"].fillna(False,inplace=True)
df["HomePlanet"].fillna("Earth",inplace=True)
df["Destination"].fillna('TRAPPIST-1e',inplace=True)
df["VIP"].fillna(False,inplace=True)
df["Cabin"].fillna('Z/00/Z',inplace=True)

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df.loc[df["Transported"]==0,'Fun'].dropna(),
         range=(0,3000),bins=30,alpha=0.5,label='0')
plt.hist(df.loc[df["Transported"]==1,'Fun'].dropna(),
         range=(0,3000),bins=30,alpha=0.5,label='1')
plt.legend(loc='upper right',title='Fun')
plt.title('relation of Transported & Fun',fontsize=18)
plt.grid(ls=':')
plt.xlabel('Fun')
plt.ylabel('count')

In [ ]:
edge_bin = -float('inf'),500,1300,2000,float('inf')
edge_bin
df["Fun_bin"] = pd.cut(df["Fun"],edge_bin,labels=False)
ob_cols = df.select_dtypes(object).columns.tolist()
float_cols = df.select_dtypes(float).columns.tolist()

In [ ]:
df["Age"].describe()
df["Age"].fillna(df["Age"].median(),inplace=True)
col = ["Age"]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[col])
df[col] = scaler.transform(df[col])
sns.distplot(df["Age"])

In [ ]:
df["PassengerId"]

In [ ]:
import re

def get_id(PassengerId):
    id_search = re.search(r'(\d{4})\w\d{2}',PassengerId)
    
    if id_search:
        return id_search.group(1)
    return ""

df["id_1"]=df["PassengerId"].apply(get_id)
col = ["id_1"]
scaler.fit(df[col])
df[col] = scaler.transform(df[col])

df["Group_num"] = df.groupby('id_1')['PassengerId'].transform('count')
from sklearn.preprocessing import PowerTransformer
ptb = PowerTransformer(method='box-cox')
pty = PowerTransformer(method='yeo-johnson')
col = ["Group_num"]
ptb.fit(df[col])
df[col] = ptb.transform(df[col])

def get_id(PassengerId):
    id_search = re.search(r'\d{4}\w(\d{2})',PassengerId)
    
    if id_search:
        return id_search.group(1)
    return ""

df["id_2"]=df["PassengerId"].apply(get_id)
from sklearn.preprocessing import PowerTransformer
ptb = PowerTransformer(method='box-cox')
pty = PowerTransformer(method='yeo-johnson')
col = ["id_2"]
ptb.fit(df[col])
df[col] = ptb.transform(df[col])

In [ ]:
import collections
c =  collections.Counter(df["id_2"])
print(c)
df["id_2"].value_counts()

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df.loc[df["Transported"]==0,'Group_num'].dropna(),
         bins = 50,alpha=0.5,width=0.05,align='mid',label='0')
plt.hist(df.loc[df["Transported"]==1,'Group_num'].dropna(),
         bins = 50,alpha=0.5,width=-0.05,align='mid',label='1')
plt.legend(loc = 'upper right',title = 'Transported')
plt.grid(ls=':')
plt.title('relation of Transported & Group_num',fontsize=21)
plt.xlabel('GroupSize')
plt.ylabel('count')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(df.loc[df["Transported"]==0,'id_2'].dropna(),
         bins=20,alpha=0.5,align='mid',width=0.1,label='0')
plt.hist(df.loc[df["Transported"]==1,'id_2'].dropna(),
         bins=20,alpha=0.5,align='mid',width=-0.1,label='1')
plt.legend(loc='upper right',title='Transported')
plt.grid(ls=':')
plt.xlabel('id_2')
plt.ylabel('count')
plt.title("relation of Transported & idx_2",fontsize=16)
plt.show()

In [ ]:
import re
def get_position(cabin):
    position_search = re.search(r'(\w)/\d+/\w',cabin)
    if position_search:
        return position_search.group(1)
    return ""
df["Area"] = df["Cabin"].apply(get_position)


def get_position2(cabin):
    position_search = re.search(r'\w/\d+/(\w)',cabin)
    if position_search:
        return position_search.group(1)
    return""
df["Side"] = df["Cabin"].apply(get_position2)

In [ ]:
plt.hist(df.loc[df["Transported"]==0,"Area"].dropna(),
         bins=20,alpha=0.5,width = 0.3,align='mid',label='0')
plt.hist(df.loc[df["Transported"]==1,"Area"].dropna(),
         bins=20,alpha=0.5,width=-0.3,align='mid',label='1')
plt.legend(loc='upper right',title='Transported')
plt.grid(ls=':')
plt.xlabel('Area')
plt.ylabel('count')
plt.title('relation of Transported & Area',fontsize=21)

In [ ]:
df1 = df.drop(columns=["PassengerId","Name","Cabin"])

float_cols = df1.select_dtypes(float).columns.tolist()
ob_cols = df1.select_dtypes(object).columns.tolist()
categorical_features = ["HomePlanet","CryoSleep","Destination","VIP","Fun_bin","Area","Side",]
float_cols = df1[float_cols].drop(columns=["Age","Transported"]).columns.tolist()
num_cols = ["Group_num","id_2"]

from sklearn.preprocessing import LabelEncoder

for c in categorical_features:
    le = LabelEncoder()
    le.fit(df1[c])
    df1[c] = le.transform(df1[c])

In [ ]:
train = df1[:len(train_df)]
test = df1[len(train_df):]
y = train["Transported"]
X = train.drop("Transported",axis=1)
X_test = test.drop("Transported",axis=1)

len(y),len(X),len(X_test)

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df1.corr().round(3),annot=True,
           fmt = 'g',cmap="Blues")    

In [ ]:
df1.head()

In [ ]:
# #model lgb StratifiedKFold
# import lightgbm as lgb
# import optuna

# from sklearn.metrics import log_loss
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report

# categorical_features = ["HomePlanet","CryoSleep","Destination","VIP","Fun_bin","Area","Side"]

# from sklearn.model_selection import StratifiedKFold

# skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=71)

# for fold_id, (train_index,valid_index) in enumerate(skf.split(X,y)):
#     X_tr = X.loc[train_index,:]
#     X_val = X.loc[valid_index,:]
#     y_tr = y[train_index]
#     y_val = y[valid_index]
    
#     print(f'\n fold_id:{fold_id}')
#     print(f'\n y_tr y==1 rate:\n{sum(y_tr)/len(y_tr)}')
#     print(f'\n y_val y==1 rate:\n{sum(y_val)/len(y_val)}')

# params = {
#     "objective":"binary",
#     "max_bin":200,
#     "learning_rate":0.1,
#     "num_leaves":31,
# }

# lgb_train = lgb.Dataset(X_tr,y_tr,
#                         categorical_feature = categorical_features)

# lgb_eval = lgb.Dataset(X_val,y_val,reference=lgb_train,
#                       categorical_feature = categorical_features)
# model = lgb.train(params,lgb_train,
#          valid_sets = [lgb_train,lgb_eval],
#          verbose_eval = 12,
#          num_boost_round = 600,
#          early_stopping_rounds=10)

# y_pred_val = model.predict(X_val,num_iteraton = model.best_iteration)

# score = log_loss(y_val,y_pred_val)
# print(f'\nlogloss:\n {score:.5f}')  

In [ ]:
#use optuna
import lightgbm as lgb
import optuna

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

categorical_features = ["HomePlanet","CryoSleep","Destination","VIP","Fun_bin","Area","Side"]

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=71)

def objective(trial):
    
    params = {
        "objective":"binary",
        "max_bin":trial.suggest_int('max_bin',20,700),
        "learning_rate":0.01,
        "num_leaves":trial.suggest_int('num_leaves',20,600),
    }
    
    for fold_id, (train_index,valid_index) in enumerate(skf.split(X,y)):
        X_tr = X.loc[train_index,:]
        X_val = X.loc[valid_index,:]
        y_tr = y[train_index]
        y_val = y[valid_index]
        
        print(f'\n fold_id:{fold_id}')
        print(f'\n y_tr y==1 rate:\n{sum(y_tr)/len(y_tr)}')
        print(f'\n y_val y==1 rate:\n{sum(y_val)/len(y_val)}')
        
        lgb_train = lgb.Dataset(X_tr,y_tr,
                                categorical_feature = categorical_features)
        
        lgb_eval = lgb.Dataset(X_val,y_val,reference=lgb_train,
                               categorical_feature = categorical_features)
        
        model = lgb.train(params,lgb_train,
                          valid_sets = [lgb_train,lgb_eval],
                          verbose_eval = 10,
                          num_boost_round = 1000,
                          early_stopping_rounds=10)

        y_pred_val = model.predict(X_val,num_iteraton = model.best_iteration)
        
        score = log_loss(y_val,y_pred_val)
        return score

In [ ]:
study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed=0))
study.optimize(objective,n_trials=70)
study.best_params

In [ ]:
import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_preds = []
models = []
oof_train = np.zeros((len(X),))

categorical_features = ["HomePlanet","CryoSleep","Destination","VIP","Fun_bin","Area","Side"]

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=71)

params = {
        "objective":"binary",
        "max_bin":study.best_params["max_bin"],
        "learning_rate":0.01,
        "num_leaves":study.best_params["num_leaves"],
}

for fold_id, (train_index,valid_index) in enumerate(skf.split(X,y)):
        X_tr = X.loc[train_index,:]
        X_val = X.loc[valid_index,:]
        y_tr = y[train_index]
        y_val = y[valid_index]
        
        lgb_train = lgb.Dataset(X_tr,y_tr,
                                categorical_feature = categorical_features)
        
        lgb_eval = lgb.Dataset(X_val,y_val,reference=lgb_train,
                               categorical_feature = categorical_features)
        
        model = lgb.train(params,lgb_train,
                          valid_sets = [lgb_train,lgb_eval],
                          verbose_eval = 10,
                          num_boost_round = 1000,
                          early_stopping_rounds=10)

        oof_train[valid_index] =\
            model.predict(X_val,num_iteraton = model.best_iteration)
        
        y_pred = model.predict(X_test,num_iteration = model.best_iteration)
        
        y_preds.append(y_pred)
        models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_stk.csv',index=False)

In [ ]:
scores = [
    m.best_score["valid_1"]["binary_logloss"] for m in models
]

score = sum(scores)/len(scores)
print('\n CV SCORE =\n',score)
print(scores)

In [ ]:
y_sum = sum(y_preds)/len(y_preds)
y_sum = (y_sum>0.5).astype(bool)
sub["Transported"] = list(y_sum)

sub.to_csv('sub_lgb_skf.csv',index=False)
sub_lgb_skf = pd.read_csv('../working/sub_lgb_skf.csv')
sub_lgb_skf

In [ ]:
# y_pred = model.predict(X_test,num_ietation = model.best_iteration)
# y_pred = (y_pred>0.5).astype(bool)
# sub["Transported"] = list(y_pred)
# sub.to_csv("sub_lgb_skf.csv",index=False)
# sub_lgb_skf = pd.read_csv('../working/sub_lgb_skf.csv')

In [ ]:
fi_df = pd.DataFrame(model.feature_importance(importance_type='gain'),index=X.columns,columns=["importance"]).sort_values('importance',ascending=False)

fi = model.feature_importance(importance_type='gain')
idx = np.argsort(fi)[::-1]
idx
print(fi_df)

plt.figure(figsize=(12,4))
top_cols,top_importance = X.columns.values[idx][:15],fi[idx][:15]
top_cols,top_importance
sns.barplot(y=top_cols,x=top_importance)
plt.title('feature importance type_gain',fontsize=22)

In [ ]:
fi_df2= pd.DataFrame(model.feature_importance(importance_type='split'),index=X.columns,columns=['importance']).sort_values('importance',ascending=False)

fi2 = model.feature_importance(importance_type='split')
idx2 = np.argsort(fi2)[::-1]
top_cols2,top_importance2 = X.columns.values[idx2][:15],fi2[idx][:15]
print(fi_df2)
plt.figure(figsize=(12,4))
sns.barplot(x=top_importance2,y=top_cols2)
plt.title('feature importance type_split',fontsize=22)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
list_k = []
list_scores1 = []
list_scores2 = []
list_scores3 = []

for k in range(3,16,1):
    kn = KNeighborsClassifier(n_neighbors=k)
    kn.fit(X_tr,y_tr)
    y_pred_val = kn.predict(X_val)
    score1 = kn.score(X_val,y_val)
    score2 = log_loss(y_val,y_pred_val)
    score3 = f1_score(y_val,y_pred_val)
    
    list_k.append(k)
    list_scores1.append(score1)
    list_scores2.append(score2)
    list_scores3.append(score3)
    
    print('KNeighborsClassifier')
    print('{}[%d] accuracy:{:.4f}'.format('k=',score1)%k)
    print('{}[%d] logloss:{:.4f}'.format('k=',score2)%k)
    print('{}[%d] f1:{:.4f}\n'.format('k=',score3)%k)
    
    
fig,ax = plt.subplots(1,3,figsize=(18,4))
ax[0].plot(list_k,list_scores1)
ax[0].set_title('relation of accuracy & K\n ==KNeibors==')
ax[0].set_xlabel('k')
ax[0].set_ylabel('accuracy')
ax[0].grid(ls=':')

ax[1].plot(list_k,list_scores2)
ax[1].set_title('relation of logloss & K\n ==KNeighbors==')
ax[1].set_xlabel('k')
ax[1].set_ylabel('logloss')
ax[1].grid(ls=':')

ax[2].plot(list_k,list_scores2)
ax[2].set_title('relation of f1 & K\n  ==KNeighbors==')
ax[2].set_xlabel('k')
ax[2].set_ylabel('f1')
ax[2].grid(ls=':')        

In [ ]:
list_k = []
list_scores1 = []
list_scores2 = []
list_scores3 = []

for k in range(10,200,10):
    rfck = RandomForestClassifier(n_estimators=k,max_depth=5,random_state=78)
    rfck.fit(X_tr,y_tr)
    y_pred_val = rfck.predict(X_val)
    score1 = rfck.score(X_val,y_val)
    score2 = log_loss(y_val,y_pred_val)
    score3 = f1_score(y_val,y_pred_val)
    
    list_k.append(k)
    list_scores1.append(score1)
    list_scores2.append(score2)
    list_scores3.append(score3)
    
    print('RandomForestClassifier')
    print('{}[%d] accuracy:{:.4f}'.format('k=',score1)%k)
    print('{}[%d] logloss:{:.4f}'.format('k=',score2)%k)
    print('{}[%d] f1:{:.4f}\n'.format('k=',score3)%k)
    
fig,ax = plt.subplots(1,3,figsize=(18,4))
ax[0].plot(list_k,list_scores1)
ax[0].set_title('relation of accuracy & K\n ==RandomForest==')
ax[0].set_xlabel('k')
ax[0].set_ylabel('accuracy')
ax[0].grid(ls=':')

ax[1].plot(list_k,list_scores2)
ax[1].set_title('relation of logloss & K\n ==RandomForest==')
ax[1].set_xlabel('k')
ax[1].set_ylabel('logloss')
ax[1].grid(ls=':')

ax[2].plot(list_k,list_scores2)
ax[2].set_title('relation of f1 & K\n  ==RandomForest==')
ax[2].set_xlabel('k')
ax[2].set_ylabel('f1')
ax[2].grid(ls=':')      

In [ ]:
list_k = []
list_scores1 = []
list_scores2 = []
list_scores3 = []

for k in range(3,16,1):
    rfck = RandomForestClassifier(n_estimators=100,max_depth=k,random_state=78)
    rfck.fit(X_tr,y_tr)
    y_pred_val = rfck.predict(X_val)
    score1 = rfck.score(X_val,y_val)
    score2 = log_loss(y_val,y_pred_val)
    score3 = f1_score(y_val,y_pred_val)
    
    list_k.append(k)
    list_scores1.append(score1)
    list_scores2.append(score2)
    list_scores3.append(score3)
    
    print('RandomForestClassifier')
    print('{}[%d] accuracy:{:.4f}'.format('k=',score1)%k)
    print('{}[%d] logloss:{:.4f}'.format('k=',score2)%k)
    print('{}[%d] f1:{:.4f}\n'.format('k=',score3)%k)
    
fig,ax = plt.subplots(1,3,figsize=(18,4))
ax[0].plot(list_k,list_scores1)
ax[0].set_title('relation of accuracy & K\n ==RandomForest==')
ax[0].set_xlabel('k')
ax[0].set_ylabel('accuracy')
ax[0].grid(ls=':')

ax[1].plot(list_k,list_scores2)
ax[1].set_title('relation of logloss & K\n ==RandomForest==')
ax[1].set_xlabel('k')
ax[1].set_ylabel('logloss')
ax[1].grid(ls=':')

ax[2].plot(list_k,list_scores2)
ax[2].set_title('relation of f1 & K\n  ==RandomForest==')
ax[2].set_xlabel('k')
ax[2].set_ylabel('f1')
ax[2].grid(ls=':')            

In [ ]:
list_k = []
list_scores1 = []
list_scores2 = []
list_scores3 = []

for k in range(3,16,1):
    etck = ExtraTreesClassifier(n_estimators=100,max_depth=k,random_state=78)
    etck.fit(X_tr,y_tr)
    y_pred_val = etck.predict(X_val)
    score1 = etck.score(X_val,y_val)
    score2 = log_loss(y_val,y_pred_val)
    score3 = f1_score(y_val,y_pred_val)
    
    list_k.append(k)
    list_scores1.append(score1)
    list_scores2.append(score2)
    list_scores3.append(score3)
    
    print('ExtraTreesClassifier')
    print('{}[%d] accuracy:{:.4f}'.format('k=',score1)%k)
    print('{}[%d] logloss:{:.4f}'.format('k=',score2)%k)
    print('{}[%d] f1:{:.4f}\n'.format('k=',score3)%k)
    
fig,ax = plt.subplots(1,3,figsize=(18,4))
ax[0].plot(list_k,list_scores1)
ax[0].set_title('relation of accuracy & K\n ==ExtraTrees==')
ax[0].set_xlabel('k')
ax[0].set_ylabel('accuracy')
ax[0].grid(ls=':')

ax[1].plot(list_k,list_scores2)
ax[1].set_title('relation of logloss & K\n ==ExtraTrees==')
ax[1].set_xlabel('k')
ax[1].set_ylabel('logloss')
ax[1].grid(ls=':')

ax[2].plot(list_k,list_scores2)
ax[2].set_title('relation of f1 & K\n  ==ExtraTrees==')
ax[2].set_xlabel('k')
ax[2].set_ylabel('f1')
ax[2].grid(ls=':')         

In [ ]:
list_k = []
list_scores1 = []
list_scores2 = []
list_scores3 = []

for k in range(10,200,10):
    etck = ExtraTreesClassifier(n_estimators=k,max_depth=5,random_state=78)
    etck.fit(X_tr,y_tr)
    y_pred_val = etck.predict(X_val)
    score1 = etck.score(X_val,y_val)
    score2 = log_loss(y_val,y_pred_val)
    score3 = f1_score(y_val,y_pred_val)
    
    list_k.append(k)
    list_scores1.append(score1)
    list_scores2.append(score2)
    list_scores3.append(score3)
    
    print('ExtraTreesClassifier')
    print('{}[%d] accuracy:{:.4f}'.format('k=',score1)%k)
    print('{}[%d] logloss:{:.4f}'.format('k=',score2)%k)
    print('{}[%d] f1:{:.4f}\n'.format('k=',score3)%k)
    
fig,ax = plt.subplots(1,3,figsize=(18,4))
ax[0].plot(list_k,list_scores1)
ax[0].set_title('relation of accuracy & K\n ==ExtraTrees==')
ax[0].set_xlabel('k')
ax[0].set_ylabel('accuracy')
ax[0].grid(ls=':')

ax[1].plot(list_k,list_scores2)
ax[1].set_title('relation of logloss & K\n ==ExtraTrees==')
ax[1].set_xlabel('k')
ax[1].set_ylabel('logloss')
ax[1].grid(ls=':')

ax[2].plot(list_k,list_scores2)
ax[2].set_title('relation of f1 & K\n  ==Extratrees==')
ax[2].set_xlabel('k')
ax[2].set_ylabel('f1')
ax[2].grid(ls=':')      

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(n_estimators=90,max_depth=11,random_state=78)
etc = ExtraTreesClassifier(n_estimators=50,max_depth=14,random_state=78)
dtc = DecisionTreeClassifier(max_depth=5,random_state=78)
kn = KNeighborsClassifier(n_neighbors=11)

models = [rfc,etc,dtc,kn]
    
def get_val_score(m):
    m.fit(X_tr,y_tr)
    y_pred_val = m.predict(X_val)
    score1 = log_loss(y_val,y_pred_val)
    score2 = f1_score(y_val,y_pred_val)
    score3 = recall_score(y_val,y_pred_val)
    score4 = precision_score(y_val,y_pred_val)
    score5 = accuracy_score(y_val,y_pred_val)
        
        
    print('\n==={}==='.format(m))
    print('logloss:{:.4f}'.format(score1))
    print('f1:{:.4f}'.format(score2))
    print('recall:{:.4f}'.format(score3))
    print('precision:{:.4f}'.format(score4))
    print('accuracy:{:.4f}'.format(score5))

for m in models:
    get_val_score(m)

###### 